<a href="https://colab.research.google.com/github/nurshatfateh/WebScraping-NaturalLanguageProcessing/blob/main/merged1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install newscatcherapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from newscatcherapi import NewsCatcherApiClient
newscatcherapi=NewsCatcherApiClient(x_api_key='jVQmu6BEi4b1-mT2FqcXuIJHtar9yWpXTEqfddfKI5g')
collection = newscatcherapi.get_search_all_pages(q='Road Accident',
                                         lang='en',
                                         countries='BD',
                                         )

1 page is going to be extracted
Total number of found articles => 24.
Total number of pages 1.


In [ ]:
pip install newspaper3k

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import files
 
 
uploaded = files.upload()

Saving dictionary.csv to dictionary (1).csv


In [ ]:
import pandas as pd
import spacy
from spacy.matcher import Matcher
from newspaper import Article


In [ ]:
filename = 'dictionary.csv'
df = pd.read_csv(filename)

In [ ]:
def extract_article_from_url(url):
  a = Article(url)
  a.download()
  a.parse()
  return a.text, a.publish_date

In [ ]:
def sentence_giver(doc, start, end):
  s = start
  while str(doc[s]) != '.':
    s = s - 1
    if s < 0:
      break
  e = end
  while str(doc[e]) != '.':
    e = e + 1
    if e > len(doc) - 1:
      break
  return str(doc[s+1:e])

In [ ]:
def location_finder(doc):
  locations = []
  sentences = []
  matcher = Matcher(nlp.vocab)
  pattern = [{"ENT_TYPE": "GPE"}]
  matcher.add("Location", [pattern])
  matches = matcher(doc)
  for match_id, start, end in matches:
    sentence = sentence_giver(doc, start, end)
    value = l_detail_filter(sentence)
    if value == 1:
      sentences.append(sentence)
      locations.append(doc[start:end])
  return locations, sentences

def l_detail_filter(data):
  data = nlp(data)
  matcher = Matcher(nlp.vocab)
  pattern1 = [{"LEMMA": "accident", "POS": "VERB"}]
  pattern2 = [{"LEMMA": "die", "POS": "VERB"}]
  pattern3 = [{"LEMMA": "collide", "POS": "VERB"}]
  pattern4 = [{"LEMMA": "crash", "POS": "VERB"}]
  pattern5 = [{"LEMMA": "kill", "POS": "VERB"}]
  pattern0 = [{"LOWER": "in"}]

  matcher.add("N_Filter", [pattern1])
  matches = matcher(data)
  if len(matches) != 0:
    return 1
  else:
    matcher.add("N_Filter", [pattern2])
    matches = matcher(data)
    if len(matches) != 0:
      return 1
    else:
      matcher.add("N_Filter", [pattern3])
      matches = matcher(data)
      if len(matches) != 0:
        return 1
      else:
        matcher.add("N_Filter", [pattern4])
        matches = matcher(data)
        if len(matches) != 0:
          return 1
        else:
          matcher.add("N_Filter", [pattern5])
          matches = matcher(data)
          if len(matches) != 0:
            return 1
          else:
            matcher.add("N_Filter", [pattern0])
            matches = matcher(data)
            if len(matches) != 0:
              return 1
            else:
              return 0

In [ ]:
def number_finder(doc):
  numbers = []
  sentences = []
  matcher = Matcher(nlp.vocab)
  pattern = [{"LIKE_NUM": True}]
  matcher.add("Number", [pattern])
  matches = matcher(doc)
  for match_id, start, end in matches:
    sentence = sentence_giver(doc, start, end)
    value = n_detail_filter(sentence)
    if value == 1:
      sentences.append(sentence)
      numbers.append(str(doc[start:end]))
  return numbers, sentences

def n_detail_filter(data):
  data = nlp(data)
  matcher = Matcher(nlp.vocab)
  pattern1 = [{"LEMMA": "kill", "POS": "VERB"}]
  pattern2 = [{"LEMMA": "die", "POS": "VERB"}]


  matcher.add("N_Filter", [pattern1])
  matches = matcher(data)
  if len(matches) != 0:
    return 1
  else:
    matcher.add("N_Filter", [pattern2])
    matches = matcher(data)
    if len(matches) != 0:
      return 1
    else:
      return 0
        
    

In [ ]:
loc=[]
dea=[]
veh1=[]
veh2=[]
veh3=[]
dat=[]
for zz in range(4):

  link=collection['articles'][zz]['link']
  article=Article(link)
  article.download()
  article.parse()
  print(article.text)
  import spacy 
  nlp = spacy.load('en_core_web_sm')
  sentence = article.text
  sentence = sentence.replace("-", ".")
  doc = nlp(sentence)
  for ent in doc.ents:
     for index, row in df.iterrows():
          if ent.text.lower()==row[2]:
           ent.label_='GPE'
           break



  print("--------------------------------------------------------------------------------------------------")
  l_places, l_s = location_finder(doc)
  print("ACCIDENT LOCATION: ",l_places)
  print(l_s)

  n_people, n_s = number_finder(doc)
  death=0
  if len(n_s)!=0:
    death=n_people[0]
    print("DEATH NUMBER: ",death)
  else:
    death="One"
    print("DEATH NUMBER: ",death)
  print(n_s)
  z=[None,None,None]
  x=0
  print("RELATED VEHICLES: ")
  vehicle = ['truck','bus','car','bike','motorcycle','rickshaw','tempo']
  for i in vehicle:
    for j in doc.text.split():
      if i in j:
        if j!='carrying':
          print(i)
          z.insert(x, i)
          x=x+1
          break
  print("News Date: ",collection['articles'][zz]['published_date'])
  date=collection['articles'][zz]['published_date']
 

  loc.append(l_places[0])
  dea.append(death)
  veh1.append(z[0])
  veh2.append(z[1])
  veh3.append(z[2])
  dat.append(date)

  if len(l_places)>1:
    loc.append(l_places[1])
    dea.append(death)
    veh1.append(z[0])
    veh2.append(z[1])
    veh3.append(z[2])
    dat.append(date)
  print("--------------------------------------------------------------------------------------------------")
  


      




  



Today, beloved lyricist Omar Farook Bishal passed away after being involved in a road accident. The incident took place in Narsingdi, early in the morning. Omar Farook was returning to Dhaka, on a motorcycle with his friend, when the accident took place.

The news of his death has been confirmed by fellow lyricist Robiul Islam Jibon, who shared a post on his official Facebook page.

Among the popular songs written by Omar Farook Bishal are Sabbir Nasir's "Adha", Tahsan's "Sholpo Kothar Golpo", and Minar's "Alo Nei Alote".

He has also penned down lyrics for two well-known Indian singers: Anupam Roy for "Parchi To Khub" and Srikanto Acharya for "Tomar Valo Monde."
--------------------------------------------------------------------------------------------------
ACCIDENT LOCATION:  [Narsingdi]
['The incident took place in Narsingdi, early in the morning']
DEATH NUMBER:  One
[]
RELATED VEHICLES: 
motorcycle
News Date:  2022-11-07 06:41:09
--------------------------------------------------

In [ ]:
d = {'ACCIDENT LOCATION':loc,'ACCIDENT Date':dat,'DEATH NUMBER':dea,'Vehicle 1':veh1,'Vehicle 2':veh2,'Vehicle 3':veh3}
df = pd.DataFrame(d)
df

,ACCIDENT LOCATION,ACCIDENT Date,DEATH NUMBER,Vehicle 1,Vehicle 2,Vehicle 3
0,(Narsingdi),2022-11-07 06:41:09,One,motorcycle,None,None
1,(Jhenaidah),2022-11-09 07:55:01,Six,truck,bus,motorcycle
2,(Pabna),2022-11-09 07:55:01,Six,truck,bus,motorcycle
3,(Jatrabari),2022-11-10 08:03:00,two,None,None,None
4,(Dhaka),2022-11-10 08:03:00,two,None,None,None
5,(Chattogram),2022-11-11 18:21:00,Three,truck,bus,motorcycle
6,(Chattogram),2022-11-11 18:21:00,Three,truck,bus,motorcycle
